In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import h5py
import numpy as np
import json
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import Image, display, clear_output
from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline
sns.set_style('whitegrid')

In [4]:
from keras import optimizers
from keras import applications
from keras.applications.vgg16 import VGG16
from keras.models import Sequential, load_model, Model
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Activation, Dropout, Flatten, Dense, Input
from keras.regularizers import l2, l1
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, History
from keras import backend as K
from tensorflow.keras.utils import get_file
from tensorflow.keras.utils import load_img

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array

In [6]:
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1./255)

    model = VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode=None, shuffle=False)
    bottleneck_features_train = model.predict(generator, nb_train_samples // batch_size)
    np.save(location+'/bottleneck_features_train.npy', bottleneck_features_train)

    generator = datagen.flow_from_directory(validation_data_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode=None, shuffle=False)
    bottleneck_features_validation = model.predict(generator, nb_validation_samples // batch_size)
    np.save(location+'/bottleneck_features_validation.npy', bottleneck_features_validation)

In [7]:
def print_best_model_results(model_hist):
    best_epoch = np.argmax(model_hist['val_accuracy'])
    print('epoch:', best_epoch+1, \
    ', val_accuracy:', model_hist['val_accuracy'][best_epoch], \
    ', val_loss:', model_hist['val_loss'][best_epoch])

In [8]:
def plot_metrics(hist, stop=50):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,4))

    axes = axes.flatten()

    axes[0].plot(range(stop), hist['accuracy'], label='Training', color='#FF533D')
    axes[0].plot(range(stop), hist['val_accuracy'], label='Validation', color='#03507E')
    axes[0].set_title('Accuracy')
    axes[0].set_ylabel('Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].legend(loc='lower right')

    axes[1].plot(range(stop), hist['loss'], label='Training', color='#FF533D')
    axes[1].plot(range(stop), hist['val_loss'], label='Validation', color='#03507E')
    axes[1].set_title('Loss')
    axes[1].set_ylabel('Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].legend(loc='upper right')

    plt.tight_layout();

    print("Best Model:")
    print_best_model_results(hist)

In [9]:
def plot_acc_metrics(hist1, hist2, stop=50):
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(4.25,6))

    axes = axes.flatten()

    axes[0].plot(range(stop), hist1['accuracy'], label='Training', color='#FF533D')
    axes[0].plot(range(stop), hist1['val_accuracy'], label='Validation', color='#03507E')
    axes[0].set_title('Training')
    axes[0].set_ylabel('Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].legend(loc='lower right')

    axes[1].plot(range(stop), hist2['accuracy'], label='Training', color='#FF533D')
    axes[1].plot(range(stop), hist2['val_accuracy'], label='Validation', color='#03507E')
    axes[1].set_title('Fine-tuning')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_xlabel('Epoch')
    axes[1].legend(loc='lower right')

    plt.tight_layout();

In [10]:
def train_categorical_model():
    train_data = np.load(location+'/bottleneck_features_train.npy')
    train_labels = np.array([0]*(419) + [1]*(288) + [2]*(272))
    train_labels = to_categorical(train_labels)

    validation_data = np.load(location+'/bottleneck_features_validation.npy')
    validation_labels = np.array([0]*(73) + [1]*(50) + [2]*(48))
    validation_labels = to_categorical(validation_labels)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    checkpoint = ModelCheckpoint(top_model_weights_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True,mode='auto')

    fit = model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(validation_data,validation_labels), callbacks=[checkpoint])

    with open(location+'/top_history.txt', 'w') as f:
        json.dump(fit.history, f)

    return model,fit.history

In [11]:
def finetune_categorical_model():
    input_tensor = Input(shape=(256,256,3))
    base_model = VGG16(weights='imagenet',include_top= False,input_tensor=input_tensor)
    print("Model loaded.")
    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(3, activation='softmax'))

    top_model.load_weights(top_model_weights_path)

    model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

    for layer in model.layers[:25]:
        layer.trainable = False

    model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.00001, momentum=0.9), metrics=['accuracy'])

    train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(validation_data_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='categorical')

    checkpoint = ModelCheckpoint(fine_tuned_model_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')

    fit = model.fit_generator(train_generator, steps_per_epoch=nb_train_samples//batch_size, epochs=epochs, validation_data=validation_generator, validation_steps=nb_validation_samples//batch_size, verbose=1, callbacks=[checkpoint])

    with open(location+'/ft_history.txt', 'w') as f:
        json.dump(fit.history, f)

    return model, fit.history

In [12]:
def evaluate_categorical_model(model, directory, labels):
    datagen = ImageDataGenerator(rescale=1./255)

    generator = datagen.flow_from_directory(directory, target_size=(img_height,img_width), batch_size=batch_size, class_mode='categorical', shuffle=False)

    predictions = model.predict_generator(generator, len(labels))

    pred_labels = [0 if i<0.5 else 1 for i in predictions]

    print('')
    print(classification_report(validation_labels, pred_labels))
    print('')
    cm = confusion_matrix(validation_labels, pred_labels)
    return cm

In [ ]:
#Input Image Details

In [13]:
location = '/content/drive/MyDrive/DS_Project/Dataset/car-damage-dataset/data2a'
top_model_weights_path = location+'/top_model_weights.h5'
fine_tuned_model_path = location+'/ft_model.h5'

train_data_dir = location+'/training'
validation_data_dir = location+'/validation'
train_samples = [len(os.listdir(train_data_dir+'/'+i)) for i in sorted(os.listdir(train_data_dir))]
nb_train_samples = 979
validation_samples = [len(os.listdir(validation_data_dir+'/'+i)) for i in sorted(os.listdir(validation_data_dir))]
nb_validation_samples = 171

img_width, img_height = 256,256
#epochs = 50
epochs = 1
batch_size = 16

In [15]:
save_bottleneck_features()

Found 979 images belonging to 3 classes.
62/62 [==============================] - 821s 13s/step
Found 171 images belonging to 3 classes.
11/11 [==============================] - 142s 13s/step


In [16]:
d3_model, d3_history = train_categorical_model()

62/62 [==============================] - ETA: 0s - loss: 4.2918 - accuracy: 0.4372
Epoch 1: val_accuracy improved from -inf to 0.53801, saving model to /content/drive/MyDrive/DS_Project/Dataset/car-damage-dataset/data2a/top_model_weights.h5
62/62 [==============================] - 16s 249ms/step - loss: 4.2918 - accuracy: 0.4372 - val_loss: 1.0316 - val_accuracy: 0.5380


In [ ]:
#plot_metrics(d3_history)

In [17]:
ft_model, ft_history = finetune_categorical_model()

Model loaded.


Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


<ipython-input-11-6990e162fc8e>:30: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit = model.fit_generator(train_generator, steps_per_epoch=nb_train_samples//batch_size, epochs=epochs, validation_data=validation_generator, validation_steps=nb_validation_samples//batch_size, verbose=1, callbacks=[checkpoint])


61/61 [==============================] - ETA: 0s - loss: 1.0156 - accuracy: 0.5078 
Epoch 1: val_accuracy improved from -inf to 0.52500, saving model to /content/drive/MyDrive/DS_Project/Dataset/car-damage-dataset/data2a/ft_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


61/61 [==============================] - 951s 16s/step - loss: 1.0156 - accuracy: 0.5078 - val_loss: 1.0318 - val_accuracy: 0.5250


In [18]:
ft_model = load_model(location + '/ft_model.h5')

In [19]:
validation_labels = np.array([0] * validation_samples[0] +
                             [1] * validation_samples[1] +
                             [2] * validation_samples[2])

In [ ]:
# cm = evaluate_categorical_model(ft_model, validation_data_dir, validation_labels)

In [20]:
def pipe3(image_path, model):
    urllib.request.urlretrieve(image_path, 'saved.jpg')
    img = load_img('saved.jpg', target_size=(256,256))
    x = img_to_array(img)
    x = x.reshape((1,)+x.shape)/255
    pred = model.predict(x)
    pred_labels = np.argmax(pred, axis=1)
    d = {0:'Front', 1:'Rear', 2:'Side'}
    for key in d.keys():
        if pred_labels[0] == key:
            print("Validating location of damage....Result:",d[key])
    print("Severity assessment complete.")

In [26]:
pipe3('data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBwgHBgkIBwgKCgkLDRYPDQwMDRsUFRAWIB0iIiAdHx8kKDQsJCYxJx8fLT0tMTU3Ojo6Iys/RD84QzQ5OjcBCgoKDQwNGg8PGjclHyU3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3N//AABEIAJQA8wMBIgACEQEDEQH/xAAcAAEAAgMBAQEAAAAAAAAAAAAABQYDBAcBAgj/xABCEAABAwMCBAMFBQUFCAMAAAABAAIDBAUREiEGEzFBIlFxBxRhgZEjMkKh0RVSYrHBM3KC4fAkNUODkqLC8RZFVP/EABUBAQEAAAAAAAAAAAAAAAAAAAAB/8QAFREBAQAAAAAAAAAAAAAAAAAAABH/2gAMAwEAAhEDEQA/AO4oiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAiKOul3prdTzSzPb9mACM/iP3R6lBIZTKqDOMmg6TRVsh/E4MDGt8+u+ywjjqKWYRU9C+Yk7YlH1QXXUB3C81t/eH1VPfxRWnaK0RZx0kqcfyBXyeJrgGAm0U2fL3rp/wBqC5a2/vBNbfMKlHiu4tYSbPCCOxqNvrhYp+Oqmma01NvhiaTjLpTjPqAgveR5r3IXPZfaNTMYNMEEjsbtZI76dOqM9oPMeWxWvXhuo6KgZHywg6CvVzx3tEozgGnmik6tIla7B8iFbuHb1Be7cyqgIz92Rvdjh1CCVREQEREBERAREQEREBERAREQEREBERAReE4WrNX0sEvKmqoWSYJ0OeM4HdB7cJ3wU5MYBkcQ2Np7uKoF0ro6u4SMjma6moSQZB/xJseN5/kCrzXU5meZHSOY1sbmsLB4mOOxd646bearVN7P7OKD3VtXWyQOcXSfbbvJ65wPyQVirvVERyGV8bBjxDwuyEjvtqpmAiXxDroG7vkP/StDPZxw7CM8ufGO8u2F4zhXhOGsbTaWiXTr0OkJB7endUVd3EEcu1NRzEno57RGP1WxDWTujbzY2xkbnHiV4h4bszR9lC0D+ErKOHLTnJpWn5lBRDU6nDV0UdeqiIwe7SxOfC/90b/quoNsFpHSii+eVkFmtjelFD/0oOLuZG6j8MBLGnZ8LQ7V9Bla80RgjZJN4NTM4cwA4+K7j+ybd/8Aig/6Avo2u3kYdQ05HkYgUSOERuga9jZ4n/d8Jc0YVg4bvcNiubJ4nZpJwGz6XeHA2DsdiN11V1qt2P8Ad9Kf+S39F8fsm2ubpdb6XHlymqDfhkbJG17HBzXDII7gr7WvDFFTxNihaI42DDWjoB5BVa8cd0lBdYaGmjNTn+1kB2Hwb5n8kVcUWKnmZUQxzxO1RyNDmnzBWVAREQEREBERAREQEREBERAREQYauTk08kuQ0MaXEuOAAFQbJwrzeJ6viS5VEtQ6fmMpqUY5QicNOXeeR8hlXi701PWW2pp6yJstPLE5ssbs4c3G42UbQVtNPBI6B2oRODdm4xjyCCHt3GNJHSxw3FzI5NZixG4v0OGcA/Q49FWuLfaHZqegqYbZXyvrXgxsjijdkO8jtstDj6CnjllY2BollqMulxglpGwGPifoFQmvr7bWPr2ujqwHEHnNB0422ONig6vZ77VXS3xVDdTXaQJY5HEOY7HRbNRmZscoBbPE7I/qFzequN3tU8NVSTRmWRgfJSPAOpvXYjGevXHyV3sHE1lu1LHK2oEcsrfHG84MRx3PTHxQWCCuliADnkrcZfRH/aOUOxmlvgkbIzs7IOUmhdpa5rWOyehOEFii4hpT96QD1K34bnTzDwSsd/iVPa2NrNToWh2cOIOQB36r2nnpJnFkD45CNvAf0QXdtSCMgghfQqB3VTpJTTu2kLWnsTspJtWwM1Pe0DzJwgnROw91qXG4UlBA+epmZHGxuXPc4ABQFxv1BQ0UlXUVcYgj6ua4OyfIY6lcV4z4kvHEMpc1scFuBzBDJIAT/E7P4j+XTzQXDjX2hVFxqHW6zVNPDSAfaPDyHybdzjAHwUA+p1PtFRHGHQSRNMkjJQXsx1AZj7w27qmUrnOofdOTyJJpBpeGHDtux9SuvezK2VPDUlVXXeSGOnqXRxlkn/CdnYA+ZzuEHR+Ff9w0bg57mvbqZrZpIaTsCOyl18NIIBBB8l9oCIiAiIgIiICIiAiIgIiICIiDHUAGGQO+6WnP0VettPyLfCwNAeW6n+p6qauzzFa6yQdWQPd9GlR1HIH0zHNOS1nUb5HUFBx7jCG+Xu91k9DkUMErI2wyYGXNacv+Hfrt1WCt4dFTQ0pjnDqrQSZQw51Dbdgz3P5Kc41u76Wnq5mUrZaalndzC15Bee3TsM/z8lNcDXaGsstM+CkER0DmljQcOJPfr5H5oOb+0WCsh4hooqUFjSxkYkxjDsgAjy6LRuDKSuhkfyWzT0hcyZzWlrJP3Scddtwumcc08F8bHA6d1MIJmx1EpY3GcgdT26dMdVWo+H2UNlnmeHBlVG5s8rDqw0kBhDOuenpkoOexXNtHM3l0MUMjXaZDGSAQtya81b5JJqGtrYZGHxROmc4Yxtg+XwWzcOGXxUXvkVyp5IAdIMkTo3O7dCFp09tuPuwmjpnSxdpIfENvPHRB42+Xt4P+21Mf8bHZ+W6m7DJcJ6c1dVcJw/XphaX4c/bfAGNzutAxHl4exzfMlpUyDQQWWhdVQvfO2XmwNY7SCW5ALjnpv0CDTor3TVemG6XS6Uk4OHmSV7Rn1B2+alf/AIma9rZKWtFbE/dsgn1Z+pIKgjTuneZZy173Ek5aCNyT/VZ6aBtLKJadnJk/ehc5hPrg7oPi68JXO2BssNTERI8MDNWh5dvgeXnutOOkkZcpTcfe4uRGZTC8kEAdS3zHTf1U/TXSuddKV9VBJVxxvBjDHta8OznOcYVhgu9l4tqY6mCme99KXR1MMmWvaH7YwMtcDpPQ9ggiqG5Wq1WQXa5UTpveDopqWoeXl2nq4Z6Nz3XsddPxPfaeoZI19KyNwmkflrWxnBLQ3GO3TyWxJbKC93qrrLlTVYfSzGnhoSwBkcben16nPnjG2VYqSnh1iENZTNbgaY2hoz/r/QQWfh69up663WmeN3IqqdzoJT+F7SPAfUHI9CrhkZwuX1s4lr7SBpFRSvw9remrt8t2n5K3U/E8dTxBb7RTx8x9RRvq5n52iaDpaPmQ76ILGiIgIiICIiAiIgIiICIiAiIg0ry9kdqrHSt1MED8jz26fNcDk4u4opKZsdW+NkBkcTGBiXlH7rTjYLtvGTpBYpWxZy5zQQ04JGVyi5UHvDGPlibHpd55LiRgA/5oKHxBxRU3KiqKaKKOlpIy0BrSdcgLvxHO527AKU4O4mh4cdJQVkkjKeo0yc1rscuQNAdkDqNgB6KLrrSypu0kWguw3UQNiFoXe0TQMbI57pHHxkkdQd9igutXxM64QMfSRPdCJWTuEwH2unGx36EDorVqdOS/myRsa06oWMDtJJ2yBjP1+So3D1JTvscFVX62UTG5wxxD5C0E4aewBB367fFRM1+tlVUv9zjuket2Q73rGPiepPzKC7VLKQvgjra6J8jHB4bJ9kHO/uny9e6mX1QgpGkQQuic9rGCNjX5yfh+q5oay0VlJT1FV75M1khjIndrDiO+c+RGy3K62tq205s1NJSnUHMMTS0O+iDovu1PUTvilihj8RAIl64Pk7IXs/C9FUtY17WuDM6A8FuM+Racf9q50+8cTUMkUQq6nJeGaKhjJWHt+IZWxJxtW01XJHNbqWZjXY10z305I+Ok4P0QW6bguPJ5XOYf4JGvH0IBWhPwlUsB01BH8MtO5v5jK0aT2iUbT9sLjTf3tEw+Wwcp6g45t73jl3en3x4Jw6PGfqgibdZayluVMZOS8as4ZIMnA64O/kpa2UdLBxHcKiooKqB0zQ2J5biNzG4/CO+e/wBO6mLfcGVZfK9sVQ5+55b2SYHYYz6fmvuo91D9Rg5WCC13LfGQe+7dkGCqvNDbpJYcjUNnjGok43J81Xa28e+T642cqIEYlmOlrfIgfqpuqkcY6mSGaOUGMuboc2TLh0BBzssN+sENfSwMltNLcRj8RdTyM8t2ndBR7hdJ9LIqF3MmfOXSTyZJdoxjbyJ7/D4rp/sep6ithr7/AHFsYqahwpmNaMGNjCSRjtlzifTCqXEvDlHY7JV3CBsuuGLU1jqrmtz2G41Yyf3lM+yDiYSMfQyRASzvBZpdsS0YPXp4cfRB1xF405XqAiIgIiICIiAiIgIiICIvMhBCcbu0cLXBwDSRHtnzyFx++8Tw0FPCySkdJJNEH6Y5WEsONs46Hv8AJda4+kbHwpXajjWGtHrqC4TPRNkcctwfhj4oFHfbVWXCeVwNPPURkHmkDxdNiThSNaaS4waGN1ggMywh22MZyPrhc3uo0Vckbvutd0K+qOoqaQiej1xau8buqC01Aq6KyS2eWYxshc9pcG7uackDPbKhaSkZT0jK0M1AtLXMJ6+ID+YXyLxVPifFVcybXKHkHGCfXCy18dSw081NBJyo4hkN/ESSf0QbFZTyR2oObAxkbHF7tsuIPYL4sN5qbFWGpoC7ltwORNlzQNsnGevxWpDdoOSIJZ5xGW+Nrh1dssVDVyT1TIW4LXyhz+2Gg+fzQdYs/FNp4lDqOpFPBKXNIikdkudnfST2x5ea2rrwxaX0Es7KalpjHuwhpbntgnPmVyarFNDcNXNIa9uWgtzvg7Hf0W1R8U3GhqXNbV1BwchjXB4B8sHsgnDZIXeExtacZLS4tP57KMuFmjphqeyRg6AvAIP+LIz8llpeIpnXBlXV1L5IwTzIOWGB48vD+q2bxxDbJ9LbXSVEZOdbZJGgn0GSMeqDXtMlvt01dUVFRC6V8LgwO2Jfg9M9/EtjhWG8UdA+6XG719JTRN+zYZ3aXnpjfbyWXhT9l3O46ZoqbnR4e1pYA8eecdfULb9o1LVXKCJ9IG+5UcZMkTJOhzs/HwH0QRtRxLeJKaSGpqoKmTOnxMjkcMnYggfmslZxhSSMZBX2uKpdGBmSCV0bm/8AiD0VWskczbzT8wksDTJucDA7/VacUbmSaHjDtw71QXe88XQ3qw/sqngrIRI+NpEjmv8AC12evXsFqcD0V1dxJBFZHQmta/WGTuLBhhyd/wDJV+kw3V/CchXjgCb3fj+0PHSeUtJ9WE/0QfoamMhhZzw0SaRrDeme+FlXgXqAiIgIiICIiAiIgL5yvpeEZQY3SYWGSpDQs7ogVrS0ZeNnYQVjj2rbUcNVsGfFpDm/AggrjENXqw5zuu/5/wCa7rd+GP2lE+N07mBwxsqc/wBjse4juk+nsCAg4tdKgQ10p93jeC7IL2g5+an7WLJt+05pZW42bABF9difphdEk9jUcn37g93+ELBL7IHRD7KUv9UHOb3TWmOSKW0z1LtTjqjneHBvpsD9Vnp5NDwS+SLI8GCNOBsrVX+yuuDDyy4ObuMKJuHBvEQFO0U+rlZDiejx8kEdI+iqXOpqmOjZUHBbVMbqaR8R8MrUq+GorfPHVx1pipyCHO0ZOrG2B5Fe1/Cd+5ut9Ly2DOlrPw/qpuhmNVbRZ7jTv95eQG4GxA3/AJAoKHXxVUzm818Tw3pyyMfRZaW3w8hskte6GXuxseoAeuQra/hu2uaZJJnwRuOw2OfhuscFBYoHlr5GyYOCJXHb6YQVN1uawl0dZC7OQctIJ/msXu00YLWvjcCME6v5eSuTW2Jz3cr3TGcfdJ3X020W6tH+zMY5+TtEdJ+iCrm20/7LbUtrZG3FjiTGQNJHwd1BXxY5JhVSycyTW9ul51HLh8c9VZp+GXxeEQSYI2dqGFgNkfSsfK3S0uO41ZcUGW1VwoWRz13IdTwwPpNPK1P5DnucR1G+XHB32HQqBqI21VRUVdK2cxibU3VHg9c7/opKpprhK0OhjDdJAP2Y2G25OMfVS09S18AijdTxjIDm0+XE79vkghLbaZ56gsZkN/E390eZK6b7O+D/AHi7wXKZz+RQkOjl0Y5r8EYHljv6qI4AtslTfjTz0VS6geTJJJK3QHu7DHku30kQZG1jGhjGjAaBgAINxvRerxuwXqAiIgIiICIiAiIgIiICIiAiIg8wi9RB4vCxp6gH1C+kQYXUsDvvQsPq1aVRYbXU/wBrRx58wMH6qTRBUnezbhN7tTrRDq88L5Psy4QPWy05P91W9EFQb7M+Dx/9JTH1Yj/Znwe9uP2JTDHdowVb0QUsezHhyOXm08VTC7zjqHj+qzu4CtrhjnT49GforaiCoTez60T04gllqzEDkMbMWgHzAGBlSVv4SstAxogo48t/E4ZJ9VOogwxU0MIxFExo8gMLKAB0C9RAREQEREBERAREQEREBERAREQEREBERAREQEREBERAREQEREBERAREQEREBERAREQEREH/2Q==',ft_model)

1/1 [==============================] - 2s 2s/step
Validating location of damage....Result: Front
Severity assessment complete.


In [27]:
pipe3('data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBQVFBgVFRUZGBgaGiEaGhoaHBshIhsYGBoZGxsbGxsbIy0kGx0qIRsaJjclKi4xNDQ0GiM6PzozPi0zNDEBCwsLEA8QHRISHTMqIyozMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMf/AABEIAMABBgMBIgACEQEDEQH/xAAbAAABBQEBAAAAAAAAAAAAAAAFAAIDBAYHAf/EAEoQAAIBAgQCBgQIDAUEAgMAAAECEQADBBIhMQVBBiJRYXGBEzKRoRRCUpKxwdHwFRYjJENTVGJygtLhM5OywvE0RIOiB+Njc9P/xAAZAQEBAQEBAQAAAAAAAAAAAAAAAQIDBAX/xAAqEQEAAgIBBAECBQUAAAAAAAAAARECEiEDMUFREwRxFCJhgaEFMpHR8P/aAAwDAQACEQMRAD8A6bdy6Fcxgayzx7SYpgvo2kK3dmDe4EmhaYxtS6o7SSsgwo5AAncfK5921SfDbu5YgeAArH4fG2vlmgx+C4e0zK9xgHY3AhVuqOYXMu1X+HYq1atlU9JcCNllrZZgTEKNNV03iB21XxXGbada5eQEfvAnwAXXyFDb3SzD/rHfwS4P9YFdPij9E3lfvvaZixwd5yxMlgwHWgnQjbQCNvZRjCJaRMw9HbZoLLOvdmysJI91Yh+meH1hLhjtNsfQxI9lVH6aCJWx4S1xp7zltx76vxx7/hNnRmxlvndB8EY/6prw4+18tj/41/prmp6X4giVtWwO9W/3uPoprdJsY2xRPBbf2tV0+7O/2dJfiKTp6Qju08eYpfhJeSXD4sf6q5l+F8cf03syj6LdRviMW2+If/Mf6FIqx0vv/knqR/0OopjzP+CT4sPsrx8adfyCjxYf01ylrV4+tdLeJuH6XqM4AnQtr3KP901fgie8fyz8ke3VG4l8q3a82FRtxFQS/wCRWQBJcR5SdJrnKcIt24Ny4c06JFvXY69TTmNe2qnGLrZhDEKRIGhiCyxqO77zWZ6OPpYzdM/C6CDmw8Rp+UG3ztaZb4sizlu2BJn/ABBoe7rVyBLh1iDqdcqzrrzHfV7DWM6Zi0cidN+4R2Vz/C9K7iIWc58y6ynGCRvaY9ouAH3GpU4i52tz/Dc/tXJkwgn1281tx7xUbYUazDAcwE90DWu8QztDsRxo+PbceSn7Kgxps3URCcqo4cK1vTQEQdCI1rkiX7tv1LjL2ZWdP91W8P0lxluM1x43hiHkD+NZHtrn1OnjnExlHeK/b7rEulLeYM2W1mKlsj6MGGUsNV9TrHWQO0A8g3S3pC1gEPbU5lDKMwYkqVY9UGUEgDNqNois/Z6auCA6I/eFdD84Zl9wom3SPCYhMt4Oo7W64B/duW+uPdXy5/pWMZRpPHqf9uu80t2eGJfvDGEg2tyLjSuZZUBVYAKCp0GoG/g3g9y2mPCYYgoQc6uZIuAnVWM65dI0gHmRpOmGF2w9uziC6MuUFWV8gO6/KA23kjKKGdGOB4nD4tCpV0Ldc6kkEHWNwQYMkDbvrEfSdXCby/bzwk5RNQ3mEe4qPkCvzjN6pyiVGhnWd++g2P4iyA5kyMyZVbKhDAkaqOfPSSYExRboyzG2+bL60gq+cGRJgwMsEkZeURQvH8NLW4t3mW4Arq7EsICqrLEgD1diTue0muP1eExGFz7bx5uoO4VdNx8Pca6rOQ6lVJhjlJaF0Ahp1jYoKIXcKrPdeXLD4oaBGUAMBz+NPbB7BWb6PcOuW74uXMpysRmHOVcAkbywVdR3zR7hRly2UAEodtdUU9Y+Y+5r2/SY45YVPPLGUzT3geCugAPOTmp0InUEHmNBppHZWhVYEU30ncPb/apa9mOEYxUM90ZFMZalNNIrZKBlrypSKVbtmnBcR0mxbAku6juhNTy6o+uqJxV25qXnzZjPKSSVFDbtwsxzkmDoB6sk65o13PdRLhlknrsAANjJ0A39barHMh2If0aDXrtoDp5nbyqq9sQM2kcgpnzMa1cwJN29/hsw0CgAz+6AIPjpWps9HLx6xA/mAHfqSC1WonyzOVMrZQZXAAiNDrJ0O81ct2QRqXYaQNAPcaI8Z4cbXVMTHxYjw7ZqHDXbZRep8UT1dNu/etxjDM5cIhaGm48gffVm1aH3n7K1/Aej9tgnph13XMEUwFG4nSSY10iKl+AWrWMe16NXUoGQPrBETvvs1TfG5pKlkAkch5CnhTyPlXVH4fZQjLaQdWdEX7Kr4bENyUAZSdABynsrMde+0NT067y5umEuH1VY+AY/QKf+Db5MeiuHt/Jv9ldLv4u8sRbVpE+vHt03qFsfiP1ST3sfsqfPPqGo6Fx3/hz1uBYg72bnmseG9C+MYVlyqywyllIPL1T9Zrq3wzE/It+1qwPTG23pXLABmZWOXbVI5+FXDOcp5TLDXliGBVjpvG3nP1UR4LnZzbX1njKCQJJMRJ05ioryQynxHtGb/bT8MGS4rglYYajlJ394rprKTNwOXej2KBOZAO7NbM7bdcGobPA8QSQo13hZbSf3JrqdhLd22lwopJUMJEwSNQJ8xQy7ggcSi2gLZNtpIEbOh1A3rltM92eYc5xWAuoIuSvODmHtBFVLmCk6NB5kqQfAAxNdkucHdgQbsgxIKA/XQXE9G8NcuiyUCsFnOgykmAZKg5TvWYzjzDVZQ5za4aGgFzvrqB59vvq3e6JPlL23VgNgDlc+RE6c4YjvNbTE9CWTrWirkcnAU+TAke0Dxod8Key4t3bRQnYOBBjmDqG8q1xPZmZyjuwr27lpgTnRokNsQO50195otgemF1DluRcA+WCD5XANPF1rYYa1bysssyNrk0gMSTIMTudiSKhHRPCXhlOa24GbqabaEkRkbfWAPGpMzE8w1jnEpOD9L7R0D5C2pVwozHaVuiVblGbU9lXuJ+kg3LQzHIV9G0AnM3Vgk5GCzoDJ5ACay3E+g160rMmV1UaejGrEnmp7pPxqrni9zDMy230ACi0V6lsAEnTQ7x6pkyZEgRy6v02HWj80X9mozr9B7o+9x8Sof0iZwwuWrkgkZDESNBmDfbqa1fD8KRfdpXKDly5RMi3akzyGg9lY3hXS625X0qqrCcrN1kEgg5bnrWzBOh013rZ8P4hbLFi2UvBE7eqqjKw0I0nzrHT+nnpxUcw3tFcjRFe14GpTXRSikRSmkaBpWvK9ilVRwL8BgDUFe8MBzEdsnx01qVsKXBtJOaAQuWcw1nMZEDTXt9tGsRwTGgqrPbcMSOY1ClteqOQNeLwbErM2lck6kOBpEADXaBXfjwxyiwfDr1u2otsVBEk2zlJ5nQ6DyNWsK+IQnLec9zEn2htPdVixdxiKq/BcwXQflE27Ka1/ETrhH8mU/QK3+VxnHIP4q9x7Za4ZKkawBuw7B31a6OcNF0WZjLEuANcqjrecD31VxxvOrD4PdE/ukjQg8h3VZ6P49sOqh7N6RmmLbRlYtp2+qfaKkz6WMZrl0huIrplXUaSdO4f8ULxjFL9h40VyhPaH7ez41BT0hs81ur/Fbf6hXnGekeGuBouFTIKzbuAzpvpHbXGMalu5mG6xlwqhIMHNv3GgtvF4lmYZtBG5HPvAFVbfSrBMBmvpqBIYNvz3Fepx3BZiVv2hMfGA2BrlETHh6Zxx9rh4jeX1kU95n6RUi8RuMdAg8Zqvb4zhT+ntf5ifbXr3MO/q3Unudftq8eYSvUrS4u8fkD51Y/pujF1ZokoBpPxS3b4itPbaNmDDxBoB0yOZAdOr7dSpJ8IEe2unT4yc8/7WCxKws9hB8p19005kkEdoq9fw2ZWHaCPaKr4e2WVW7QD7QDXq8uDedDsc1zDoMxMGDMbtqQO4HMKOeiPwq1qRKONP5TWT/wDj5gty5bIHyh5ww9/pK2rL+cWT3OPaorzZxWUtR4+6+cL/APkf539qEfBvzyMzRl3zHNqO3s0rQUJuJ+dof3T/AKT9lcoh0zjt91o4Bfl3P8x/tpXOHW2EMucRBDkt/qmrleE1KaqGI6RcC9Eq3MPm1cAoSSACDqp3HgZ8qiw9/LAuQHGoBncamDpJ7CNNDRPpBxJg2QaIsEtEySeU6EaEEb6z2ihnS/BLbt2SirlBKkbyQAVMnUx1t53r04RdRPl5c6iZnHwJjFBkKQWB1zKYcHtGbflzB0rPX+D27pJIDQYzah9ttdQdtD2zQ+zxK4NjHkKm/CjEguf5o1j+WCR3V1joTHZxnq33BON8KNgo85kYm2SdCM0ZZjQ9YDXvNSYQ3sOnpBcVU0LK+qEsRC5ZzFtRJAEbToaN8Z43h7tl7Fy4hbKGGjK3UMjedTqBMb1lePcWuMqWXdMqgjKo9aW65hj6xJMsOzbtxPHd6enNxw23BulY0GYJrEMwZG7MtwbTB0PORqQa2WD4xbfRjkbaDtPcdq4vwzF2luHDAItq46uG+MjuqCMwUaSACDJiiGD4jdw7FAQ6AxkbVd91O6z3aa6g1z4y7uvbs7Qa8zaT994rC8D45bvQtq41t/1bN/p5P9PcKMNi8Qu7ZvFR9UVicPTUS0tKs6vHbo3VfIH7aVZ1lpSx6R6NuS3BP84a2Pe61K1qjeK4ely2yRlLCAw3VvisO8GD5VDgcly2GZAG1DLr1XUlXXyYHWtbs0EixThh6OjDW/k0jZtjWAI76u5QH8Hrz4PVPE9NMErFUDvB1ZEJXyJInxE0Z4VxG1iVzWsrDnuCO4qdQanyGqn8Grz4JR42FAmK8Fteyr8iagDYEcxUTcLtne2p8VH2VpRh18KiuLbHafCnyGrNtwOyd7SfMX7Kibo7hj+ht+SgfRWmR7Z7fZU4w6HmKfJBTHN0Wwv6pfIsPoNefiphf1Z+e/1mtk2FWonsoNM2vgTTdaZA9FLHIOPAj6wagsdE7cFRcujKxGjDYww0jsYeytwMJ51AmFK3Tpo6T4FG19ocfNpumrJ2OjXo2zW711W7QwnSe7vPtq0MHiQQRi7pI2kId9+VatbK91R3ryIJB25CNak5x5NAANjeWKbzt26aVxuYN8KEjtsp4cjRXB4sHR1AnUEbSdxHKDV42kI29hqbxPhZxBBfx36+2fG39hr25fxbKJuWyQNoZQST3SdvZvrRk4Ucq8OFpcFMNieGY1zJNg9n+J5e6PZTMTw7HOoVhZIGwD3O7tB7K3ZwlNOFre8+2fjx9Od/gTFj9FbPhcP1rXv4OxSg/mqE8j6VY8YIE10I4WobuHq/Ll7Z+HD041iuj1+3mY4UM5k5muBiCZ1AzjXXsovd41w+FzYG4SB1pt20zGNTLOG310FbHi+EJUxJMVkT0Ye5cEwZ10mASJgzzrE5W6Y4xDJPdR3f4NZeXbqA5mK6yNY1g7eAq/gOHYhNXtXmJEf4dw6ezvrofDOjS2tQATRgYeOVWOCeXJGwl3MYtXQQ0j8ncBE9YHaRrPsrpPR3i104dfhCtnBKy2hZR6rMOR5eU86tPZi4h5OpTxZeuvu9JTnwvj7Kt33Snr8RT5NeVXOFpVeFbMGhlweiv/uXvdeRf9yL7bfa1EZqvj8MLttknKTBVhurKQyMO8MAfKuSliEJQgGCRoSJHmOY7qA37d9QfyGFc7jUry2j0ZFE8Bi3uW5YQ6kpcX5LrowE6kcweYZTzqS5zkedSxkLfGnEq+CsoR8UsV/21e4Ylxpe3g7drNu3pHQtEx6iyQJ8NaNm5FL03fUtaTYbMqAO2ZufZvoBzMDSecVMHqp6SvVei0nd6qNcmpLh0qICsysGOYqexeqG4KjtPrUBItIrxFqo97LStY4bGrZS5YxBUwamxDjMjD5UHwZW+sLVHEOPWFOxz/kswOxVvJWUn3A1bSkaXiQUUFnBIOqiNSJMkb0H41ZxPo89qwGbtd0CKObGG18JFE8ThEdyTO8ghmU6gTBUgx3UP4pwMugFrEXbZGsG47Ke4hmJHiKzx5XnwCjDcSZBmtWnUawgAPflYEj2zVu8ly0AcziY0IdSCRtroT4E1PheEXgsXLznvW45P0Cr+H4fbVg03GI2z3HYDvhjE98Uml5DbXF7o+MavWONOdzUuKwCnUCKFXcKVoNFY4jPOraYmayCXCKIYfGGlpTRemr0mRyihSYmakF6rcpS6yqeS/fyqEYVRG+hncRVdrpqu2IOwptK0JtdQdlK0wfQLI7TtVbDYEt1rm3IdvjRFTGg0rUTLMxAXxS0FVWHxLiHwUuqv/6M1X8i9lU+Oj83vEb+jcjxCkj3irU1dkpIEHdXlIHzpVdikbYpBXv4QtbZhPloJiTPjXF+jvSvGZGe7cN5ROVCqlnCqS6oykEMOq2s6TANabD9L8MxFtiLbkwoMFWkmAW+I2sFWAOgqcrw2uJvJbf0wICNCXR+9ICXOrIETlY9hUkgJRMmQe+sWhvgMnoWdCMsFTGU7iOYM0X4JfuKgtXVZXUdQtuyCACTzYSAe3Q7kgSbOFy+sVUNyr1xpoVjJFZaWVvipUuCgXpdamt4mg0CNIp5WhWGxQ51et4kH7igVxKG3HytRpFnl9FVsXwxnHVgHvP2UotXsw9Xkv2k3Kz5UCu9HsWf0iAdzN/TVR+i+K+XbP8AMf6aVJcDuI4yh0EDs6szUN+6WsuIMlGAMaSQYoK3CsZbEfkyP4j9Yp2BxVxEKPq2bXKZ0I0+g0qTgWw3EFZUbWGAiRodBVsEHnFZfg2b0KTHqgAnlB1A86OJdmsy1CwLompF3qi0zRPD2+ZpBLxqqYi1NXXIFRiKqAOItRVcXIPPQSYB0HbRjFop50Ku4c5p0I5gzrGonuoJ7GImKvW700ItoVAFO9PFAYdiSAokmr2GsKmp1bmfsoBa4kV5VKvEswnUeNWkmWgbEUw4kUCONnmdPGlgrnpGJLQB95PnFaQXxL57br2qy+0EUsJdm2hkeqpnxUT5VWuYhUR5kdUyOegM6Hl31VweKlE/hHtganvoDQuGvKE4niISO0/RSqFOUY3GWrmYWwULFWLGZlBvp6r6kFhv3Saz2M4ey67gzvqDrGh50R4jwu9h2i6hHYeR8DUdvEQCDBBEH7/XXSmR/oR02awRh8Q59FsrNvb15nmn+nw26bfU3AMrAMpzI3Y0e9SCQe41wnH4QRmU6HnGx5r3wI151p+g/HrjRhnulSoPoyRMgfo9+zUdwI5CszCw6lhsYLg7GByuvNXG6n6QeYIPOvbpkVknxdxLpuZ5OXK6hSM6jZwJ9ZZO24MGerBixxRGA6wE6jvB5iucw28xdqNRVH00UWUZ++or/DCdhQDhiu+nJxEjnXl3hrjlVV8E45GgLWOPstXk6SVlTg37DXhwlwcjQbA8YR9wDTreItzIABPPwECscli52GrFtLg+VVuUqGquXA2k0HbClbh1OWA08tyPv41HYe5zFWr10qslgdPVjbUc6kKF4B4SQfVZ0jkYuMJ93vqQ8RK9lVcLItuT+sePNyfrpoVzyNLIgWw3HFnrqfECr78cRhAJHurLvhrh+Ka9Thl47KaA+eKmdxHLt9tNPEiedDLPA75+LHjVuzwK/wBlA+25J7qImIqPD8Gujcr7anucKuER6RR7aUWoYi6o50MvYsdtE7nR0ne63kh+s1A/R22N2dvNRVQL+FTtUgxBHI1Yu8OK+qg8SxNUbiODrHlQFcES2rg5doEk+4feauNxS0gC5QCJGh0E9h5gzNZ22zgyDr3668jHaKt4LBKDmYyeWn0zvWke8RxZ9E5kmEYyeYymPqotmt2xqdh2jlyobxTCA23OZiWZREL8dwgGn8VL8CtoWzHnJgaUlEWKxYnMSNTzPupVi+nF0NcWwhIVBnaObNIUadgk/wA4pVNS3SOluNw9nDPcvqHWIVObudAo7PGuYca4TbS2mIw7l8Pc9Un1kbnbfvH35Tf6Q41OI4k2zcCYeyCAcwGZ2EZxOhEggdw76H9BrmZ7/DrhBW6GCcwt23JV18Y85FbvkoOwzj1W2Oh+/d9tDsQWtXA69Vlb2EGdfOrd5CrEEQQSCOwgwRTOJrmUHtGu+4039lWUd14JxDC3bVu6EHXRWy6mCRqNTvMjyoNxLEph1a4LGe2WkiBNvMSWZmP6PYD5JmdDpjv/AI74kTYa2T6j6dyvqPfmrZLiqxbUQbZ6a4dRpYHldWpvx7T4tgDxuL/TWU4l0cUkthyE7bber/I3xP4Tp2FQIoBiLb2zFxCh5ZhAP8J2bxBNbjVmbdLHTgHfDKf/ACD+ikemSfsif5g/ormPp/CvfhPdWtcUuXTk6ZAf9mnlcH/86c/TGf8AtEPjcH9Fc1wjG4Sqxm3A5nXWNIJ7ufKnY13tuVZGAkgEyM2XSR2j7alYly6OOl0kfmlsD/8AZ9lupE6XjnhF8rn/ANdcu/CPj7a9XiXefbSsS5dRPS23zwn/ALL9aiqWJ6X4RXBa06Hmi+jI7jEgg/eK58vFj8ph5/3ocjkmd5+mkxiXLrPC+kWGuF3TDO4zmCRb06qaQzfeaJ/h62P+2I5am1/VXLOFcPvNbe5bTOAxIAYA5soGs7Vat4S9dC3FDXVDZQzZF9UkG3DfowZ1jWNa5S26knGxysP5ZPqNS/hMxJtP4dWfp0865hw/hV2+jG2FgypGcLEjUCSCN9DU+H6PYkdVRlA+W4H0mTS1p0McYP7PdPkn9dP/AAwdvg1/5i/U1c/HAMQgLtBABPUcM23xACNfOmYfhGJcAIlxREk3mMnaNSzSd6WlOifhgc8PfH/j/vTfwyn6u8PG21YReB4xSplRrA67bwTrA0Gm9eWeCY4dX0jGBq73HaeyGJJPsFLKbpuOWuYuDxt3PqFRnjWG5l/8u5/TWPThWMBk3hA5B7knu2H00zC8Ox7AlrzqN5a4TGvqwgkRI5bc6WU1z8YwZ3cjxS4PpWq1zGYJv0qjxB+sUBXA40f935B7v9NeJh+IEnNilUEmOu2iyYEayeU1FE7i4WRlxFr+Zwv01KThwOrftt4XFP11lsXwi8t0XXvo8LJcsJUrMQCc2kDQaGTNPxtq82hxFhh8osF3/dUQPf41pBrG31IRQwM3F2IPqTc5d6CliOIP6s8orLWHy39btu4Et721VQGdoglfWICHX96p8ZxEKrNOwJ9goMpcvG7eu3DJzOYiPVU5VOvKAKVRcBWV57A8vrpVWXTrXQnh1y2lw23XOoeMxDdYAwQANdaXDOhODt3Rdti4jo4KkvPqkESGU7wOfnWU4d/8mMFm/aV20jISpPaSGkCPfWx4B0wtYkOVt3UyIXcuEChR+9m9nbB7K1wObdJkAxN6P1je+SffQzEH8mO2T9Xu+yrHEL/pLjufjsW+cSY99U8WeqFnkfef7VJ7Al0MuBGfkGUT4gkj3Zq1nw0fKrnOHuG2ZUmRVw8Xud3v+2sy1DdjG99SDHwIDGDuNdfEc65+OKXO0ew/bTkx9w6A+JjYdp7BRbbVzZJk27U9vorc+Zy60+zhbThiLdsBRJ6gGg3iN40076BYTCXAT6RjmRhKLlJYHVQYMjPsIB7TApuI4yyghSGZTFt0WAFJzSZJEkEqVyjnJNZsaXLhUR1yAobgjIYz5RMmG9X6JFDcR6B2LNbUk7nr68uTanv51k7uJuMSc2pM7Aaz2AaCkLzjczTYppDYw36pfbc/rpvoMN+qX51z+us495uTez+8003m7T7aWU03osN+qX51z+uvUXDDa0vzrn9dZgYhu33mmm83afbSxsbWPtoCFQAHeGua/wDvXi8RRRlFtY7JePZnrIM8gFWPYRIJJjUxyE6d9Mzt2n2mg2yccAiFURtE6eHWp/4xH7z9ZrCFm+UfaaQDdp99Ebr8Ym/d9g+umN0hbsT5q/ZWHhu017lPfQbNuPt2J8y39a0w8eY8k/y7f9FZSzhbjzkXNlXMRIHVBgkTEgc+yio4eMODcDI1wQyllOUQGJCkNu4kq2hPo2iJBrQ0N7GFUzP6KWUlB6O3OYFQVaUgEZjpvpT8Z0hFssFKFoCkC3bCrAPqkCdyIHxTm3rFY/iBeUtAqhAzDKBnYEnOQJynYaH4okmqSKez3URq240SSSEk6k+jtyT3nLS/DbDmg/lQfQKy5tnsprK3afaaDVHj7/rI8DH0U38Y3/Wkfzn7ayZt9sn305bPdQak9JHO92fFydPbVPG8YL23X0kyuWMx1zabT2UIw1lMwDSV16qkKS0dUZm0UFokkaCmYnDsrkMwfKYlSCNOwxqO+gvcHvAFp7PvzFKhHW5UqDXJ0eunEPZRfUcqxEKFE6Zm2URBn2AminH8bbw1k4KwwZiZv3BszDa2vcPt5k1U430zvXiQsWkJmEiZgA9eAeXj31lrl2dBWqiCZPJkxVfEPmbTapkRjooPee2ruH4Yx5RFZnkgLS0aeLB7K0Vjg7GJI10FW04PptMc4/tTWS2Yw+CZmA0EmJOg17Tyo9wzhVy0wf44OUqpUlcw33gsBqUbcSRqDBbh+ECMTAggiSSCAdyIEzpHIwaficGbhhZVSAus9ZV1TNqZI0jWYjU01ktn8diWkrbJBBIL5pMZjK23gN6MmTDE70MVGGw9n/Faq5wYaTE6e/uG1JOEzIggjTY66/fWpq1bKvaYmSST3zS+DtqSPqHurUpwxTO898x7Y2qe3wdTyJ05Cf8AimhbIDCt2fT5UhhD2VsxwKYgQJjWdfOnJwQagchMco+/ZNXVLYwYQ9lL4Kez6K2R4IYnKO3WYgxz5eHdU54EsLI39x7h2ez301LYcYU16MEeyttc4Efkt5TH38aiXg77BABO7A6fb5U1S2Q+AnspJgT3+yt7Z4C0wQBp2e+rC8C22MdgGx/vO/8Aw1Lc7GCPbVqxwd3BIjT1pIGXvM7DlPbW+To+p3A5aR7fv41fs8GRR1V1HdoSTrMDUEaaz3RvV1LYtcI1m2VJfKrBsykSSSNlJIzDmmqsuo7SHxSNcgQVQeqgnKveASYPhXQ7nAZ3nQR4LroOwd1QNwaD2iCNcsDQxqfGrqludDhehbLp4U5uHmPV+n6K6KnBBJ2A78vf7fvtUr8ADch4k6baCKaluYrwppmNZ++xr1ODNIgR5zPtrpT8CSdSgP8AFzHb2bj201eDJoMwMzEbxCnTvpqW58nB258uwffWl+CJ2Xu0+rviui2+EoI2MgdszpsBIG+up1pPw9QAJbfmpHLQSRyjerqW52eDt2HWSNBSHCHA1Bgb8h9/srpNvh1sAgMdDyBOk6DvO9ePgLfINoPDnHhpV1S3MrnCW7NOWoP0Uq6A+DzaLaYnfnse8DU7Uqmi2w1nojiD6zovbBZo9wolg+hXWGe6SN4Agcq09jimDA/6hO09arVvjmCH6ZJ7S1XUsPwfRa2oGmvfp/xyon+AUAhWymPkgwe3xipR0jwX662f5qdb6SYMfp7ftFJjwWcnCUiAGOkfT3UrPDTKiRB3InSdpkxUv41YMbX7fz6q/jFg+V5BrMZzE9piqliicFTYxA10Hu76avCbbaBpC6GDMHfcTrrUB6TYIiGxFojb1jXicf4eohcRaQSDCmJIIOseFSbXhdXg1sdvkY27KiPBLYAAmB3xPdJE9uo113qI9JsFucVb9v8Aak3SfBafnNvyP9qFprfB0WSNyInSfDUbeNTDh1uIKz9+6qa9KsF+0J84V7+NWB/aE+cKUWsvYVTlCiBEAlp17O2pFwqEAkASNKFP0jwTGfhFsEbHOPqBp6dJsEoC/CLfjm86UWLDCJvse6Ps7q8bCqd5Phz8e2hn404LT84tntObnTvxswUkent+JY0BM2B3785pPh5mdZg+zx23oV+NeE/aLfzhXh6VYPT84t/OpQLKgXnHLbt/uffT1tCN/uaCDpThJ/6i17frrz8ZsHP/AFSebf2pQMPY8Oz/AJpgQbEiezShP4y4T9qt/OFN/GTB7/CLc/xCqgm2GOnWI7tO7u+8010tgw1wBnkAE7kDWBzNUB0lwf7TbjszV6vSPBA637ZB/e2NOThdVHFwJ6MZMhly09fMIXLzBGs91PgF8oKZ11YHSRy23ql+MeB2+EW4P731iom49gM2b09rMNjPdHnpTk4FWwrzICjaTPZUjWjzI7yJE6d1CW6T4PliLc+I1pL0mwUaYi2O0T9FOQSS2Au4I5T38tBqda96ogGTOsff6aEnpJgv2i3HZm+ikekODj/qU8M/1nxoDDKJ5Ec/W18I0OtJnAEKo8/uTQT8YsHzxNs94P0xv5V4/SLCftKfOFAYuXn2keY/tXlAH4/hP2hPaTSrXA5TRzAdG7jhGuXLdtblt7iZriBiqW7lwOykytslCC/KdeVA6OWOkt1LHwdEthMrK0m4057dy2WCs5VGi4xJRVkxMgRWWTLvRrEqwVhbGZXcN6RMpS0iO755jLkuIwPMHuohxDoXcQkJesuRduW49JbWFsorszyxywpJZd1AE+sKqXelN1rfo2t2yRba0jw+ZEe1bsvlh8pLLbQywMGY7KlXpheFz0norQf0jXJAdTnu2xbukFXBXOoBMHRgCIpyIl6J4okrFsamJuIAyqi3GuIZ61sI6EsNOuBvpTvxVum2lxLth8/pC35W2FRLLZS5fNBSd22GZRzpl7pPde4lx0RjbLsvXvgg3Vtqfyi3A8gW1g5uZ35M4j0juXyDcS2YuPc0DrK3CC9psrda2xUEz1pHrU5HvBejly+bLF0RLrhAc6ZyucW2dLZMsqsY8aoY/h1yyYuZVeFJt5gXXOpYB1HqmIkHUZhO9GLnTC6zo7WcOXtsGtsUebYDq+RRniJUCSC0FhOtBuJY977i5cC58oV2AILldM766uRAJETE7ySFSlSpUCpUqVAqVKlQKlSpUCpUqVAqVKlQKlSpUCpUqVAqVKlQKlSpUF3DYW2ygteCHWVKMY7NRpr7qT4S2AYvA9YCMp1BCktvyJI/l76rW7mWdFM/KUH2TtTxiT8lPmigcuHt/rANvinnvz5Uy7bUAEOGPYAR29v31r34Sfkp80c6Y92REKPBQPfQR0qVKg//2Q==',ft_model)

1/1 [==============================] - 1s 1s/step
Validating location of damage....Result: Side
Severity assessment complete.


In [29]:
pipe3('data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBwgHBgkIBwgKCgkLDRYPDQwMDRsUFRAWIB0iIiAdHx8kKDQsJCYxJx8fLT0tMTU3Ojo6Iys/RD84QzQ5OjcBCgoKDQwNGg8PGjclHyU3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3N//AABEIALcAwgMBIgACEQEDEQH/xAAbAAABBQEBAAAAAAAAAAAAAAADAAIEBQYBB//EAEwQAAECAwQFBgsECgECBwEAAAIBAwAEEgURISIGEzEyQUJRUmFicRQjcoGRkqGxwdHwFVNUggcWJDM0Q2OTouHxc7I1RFVkg6PSJf/EABoBAAMBAQEBAAAAAAAAAAAAAAABAgMEBQb/xAAqEQACAgECBQMEAwEAAAAAAAAAAQIRAxIhBDFBUWEFE5EUIjJSQnGBI//aAAwDAQACEQMRAD8A9HBBoHKMPRB6IwBo8g+SnugdoT8tZ8m7Mzboi02NRRnRoFnJmTkZYpmbJttod4iuujFT2nhGdNj2YJDyXJoqa+tBRFW6M/aFoTmlM/r5jxci3mZZLYCdJU4qvBOHuOLzUoFMoPlEW1YdJcz0+E9OlmWp7IsU0i0mdzA1JN//AAEvvVIattaUcgrNq6JSyp7UVYqiemjOrWkP5lu+UcB2czap+ouSVSLddswVITl4PRXpGNurAzelFvy9rlMGxJE/LyqAQi2pIom4mxEXbeMJr9JdsAdJyMkRdHVki+i+KGQtOc+033yPxqkDdRCmy9xbsE6ki2btmqrwhgtZVvExWNybMcF9F8EpJczlw8Apx1VtbXwWQfpStEP3tiyhfmJPeiwRv9JEs6f7XZRN/wDRcQvYqD74jWfNWdaAET1mMNiJZqWx4c2CL6U88TW7NseYPJJ6urlNuLeqd2yFzRUuBxJ86NHZduWVa0sJS+bVlmF5tEShb0USuVUuVb9q3YrGF0z0MbsGfGes86rLnBXUZr9Wt2LarxuTYvMnVfEa3rN+wXm5mzH3tQ5hrMLwXiKrdii/CCJpXOTsi7JWsITMpMXK4QggmJJsNLkuqTnVF9ERKO2xzZfTZTV42mU0mDVdJ0lSObmwx2cYMUw0Z6o2iHlCQ4EkWn2XOSVRS7DkyJXEw8y0ucVuVFXgmC7OC3pAhs22HWS8Is6bF0SykLaqt2xY43ep2eHKE02mmNKbIA1obtOYiugSTEtMeNaIdaI7rl/swiW1o7arstqPAXCy7zjapd3psWGFojatYiEs+PAnBHHDYqJzRFR6kOEn0K9J58Ka2mc2BZbrr+ZYFOSLrrzFZNkQ7pEVyXbcFi/XRie1NPg0yRcqocO+/ngrWj1sakv2UnBHEahWr5Qe5W8QUJ3sjKLZk5XyavKxXDhzxJlLN8HNvwgR1g8kcdqcyxqBsO0zP9w+LX8wacbuZFXZDH9G5wzqak5kariKrHC/ZA803sylDI96KRWxl6qJkny5WalOrCBWijQaivxlQ7tK4Lx80aiYsKcdAaLOcbKq+qlL702X9UQJjR+1fA2mvA36sd0UXr486xEZOTTYaJdjlksysuyLVLjxEN5ERXDft9kJLckwnGmJeZ1eakmyxRcMLlTZjHXLOtjwBpqXs6ZbKlKvF3Jfxw74py0dtrkWc+JCV40jtXbivwgjg1W52JRl2JLk9besK4D2ry4URfA9J1xKzZq9duVY5G30/wDQ9Eux7TrRAKjKkR9iR5vb1rlpHaWqaL/+fLll/qkmCkqcUTgnFYlabW4Tp/Y8i7mL+JIeArsFOtYnaGWQMuyM86I0j+4ErrlXpY8E4fSx6fLc9PDBbzlyQ5NH7QFkWgabbaG5aSJL1vS+9ccFx2REnLGnJfMcnrKei4i+lEjSTClMPVNWjMj0hZJKUK/FVVRXgvOqYRI+0CBmrVCVI8olXrXHBUXzbV80Q0d8PVMkdtKowIuV8mns0rentiVLLR+YV5PNtiz0kqOTKelGm9aOJZbr0v2qiLt237PdfjDtm0HWnCDxZCCrlFMOHG+7ZGblWx7ODMs2P3IgbDY1tr0/++RPQN6+9Y0toSDZzLghMkJFkJtsVOnFLrxRMMCv67oyOjM6+E+05qheMnTKor94sOGF2HNHoLhsVi66TgiVyiOtURTZsRLr8UTFebzJ5fqWRxyKuxxYU3iry2ZoEGxza8LLw2TG+oqaTax4ouNyoiqmKpfhGiKUopJotY05i251Ye34e0VoS7E3LEOs8IYpzNuFUqYXVCq3qipwvvTqTbD2rUkZezSk5ciedlbg8YKjVSlyLfcqYonBeMPhOLtb/wCg8clKuhLmLOG0LKfljHeFaah2FdgvpjzcFPVi3q1EhJUpLgqbU/5jYyGnLATJVytTGFJCQ1Jhsu44ptS7BUwjP2k+1N2rMuWWJXTDt4Co3YriqY7MVWPSbO3g7UmnsqPT9BhL9XpZp2oioVQq2oFS3JhF2YugfZjLaPzvgMm3MmJKy4WoY51EAFKrk51Ql88XS6UNbuqL1ViGfOcYqzS/stHBIwyZSgLK+OpdEvK4RATSZijOLhflhfrIx9056sI5S1dadzE1mGqHNB08pRVjpM19056sNLSZrkMOerABYgha7OOWpc0OeazjQPKisDSNjltOerDv1klvunPVgGTH0GgqBqKlYeMtrQqzDliD+skt0XPVhDpJLf1PVgESAlXQzbwwvBCOkjESHGIw6RyYBSAuRXLpa/XUFnOdnxgwqGWasrevi45FX+t81/6UX9xIUFBZ5/o9ZpTs4TsxUQiSm85xNVXYnfs6o2iNu1i7MCWqyo2yO7js6rkRFVVwVdnEoxOi9uvylt6jWi209eA1XXNkuCF7ERV5lWPSmmPC5MqCpKlRq2qBUqi34JsvS+5F79kdeqzq4qDxtQ6HZadrOkKRp+sObzRKnGidAaOVlq444IirxRb7l7++IstZ4/vZgdQ6O8JEOF3n2R2ctBqXDPUQ4avnW5cVTHzd8I5SI8PiZlreqbJRqwzXLwXrRMF8/GPHp96mWdzZnC78E/5j021p37PsqZnHSIXSEhHnqVFRB8yKq+jnjyS0VoZTyFXzrEtW0erwWSWPBkl0J+ijJFaDKBvUX+m9V9ixrbWs2cdMaMw7POnC/uuVOqM/odTLz+tPdbG7uwuvu490bdu3JaVqA3RHV5cwkSbEXBRxux2KmGNy3R4nqEsiz3BXSO3EtOKKZWSchOS8sROlSIipFVwRExWKdioGZmZdIqiz5ruH0ieaLmfn37YPUNeJlhLMWrJFcXgON6r3cb9nCKa2ZvApGSESLDWlUlyIl2VFvxW/b3J1xHDRnKVNbvn4NY/c0+xRyk1RMVByUy7OK7Vx6osFnzmAInaeZSEePekQAl0Cqp4c29Teq9ywVtREBEN2reIkFFVV5o9xrsa4c7xL/pJLmb7RZ512XYdddzFMgjfNcjRCqInnT2RrKe17vlGX0Qsh8wanHRfbYl2yUarxEyuW5BRURVwVb1xRb9saS/yoOh89x+WOXM5R5Bbu17vlHUQul7vlAVLyo6h+VAcIbN0vd8oSIXS93ygVXlerCRfKgGFzdL3fKO/m93ygN/lRzL0ygAMol0v8U+Uco7X+KfKBV9ooSr2ihAFo7Q+qPyhU+T6o/KBIQ9Io6LlGar1hvgAfR5Pqj8oUO8K7Lf8AbhQwPBjUa+SPpja6NaaeDUtWgRVDcOu3r0TBEJON3Oi398ZAnM5eOLe6vnDCca6Ql9d0CUuh9JlWDJGpyR7KzpHJzAf+Jywj0SuwxRbkQkRU4+iK209ILHlDJ2YnPC3cF1bOOKJhiuCbOdbuCR5ZWPIH3/G6DMy8y7+5k3XPJbVfdfFfd2OL6bhE95lnb1uP23MgZjqmG8G2R2InfxXrjO2olTzY9K73/KNBL2Lbr37qx5v8zBInugTmituu2qxKuyerfcbJ0RJwEyIiopLeuCXqiY8Vhxi73N+J4jh44PbxvsRpJlfBheRzVql61VXLf1eaHtzsyzyquyQ/JUjSyn6O7fIBEikmh7TxF7kVIu5H9Fpf+etPL0WWviq/CJnijLmU/UuHUEknaR5/9oTztQk6Wa+qksVv2oq7buq+NFYuhVsWwAuvUyUmW6TgrinUO1fPcnXHpVkaGWLZNJNSuvfHdcmMyovOibE9EXqNwRjGP4qjzsvqOSS0wdIxVm/o3sWX/i/CZsv6jlI+ZBuW7vVYvJaxLMsyZHwGzpRjZmFtKvSuPti7UaIhuFW9BJnEpNvckzbdf5m09qXRlyc+sI1ji1g12finzvjMvtsAZfsusKpfTevXGTddLCrBawfokhI8McUR/Aj6y/OOZfwP+Sw9T/UVeQuuGEr4wK4fwP8AksPEBo/gx/uLC1P9WFeR6Oj0Yarow1QH8D/9iwNQH8CX9xYNT7MdeQqOtdGOq8PagFLX4Ev7kPVtre8DL+5Brf6sNK7j9aPahE6PSgCoxR/Buf3I5Sx+Dc/uL8oep9mKvJI1rUKIurlvwr/rf6jkGp9mOvI1uxLK5FiyH9hF98Sm7Mkw/dWdJD5MsHyi2BkYKjXZjoJsrW2KNwWx/wCm2ie5IOKO9IvWicjUNmSalJZ1+YIW2mxqJwsERIQrKy0ZpqzJB2cniIWmxzc68yInFVjuiljv0P2razVM9P3Fq/uG03G+/G9eteqItjypaTT7VsTrRDZUuVUhLuYK6X3xJzdFF7+/Wm+IcqB7bCHC2IR26BNmTp9mDLCEcWAuviEEOIL6QDQGZmyiM0/WcJWCdODhZ+q/mkJdoUVImbRpEtGlrlvR71T4xlrUmXZeZKgRpx9N/wDuNOwo6mmrlJu8cdvVFHOSzExMkMwRCQihZeu/5RlqSV8wq7KpLSd6I+tCS0HeWIxO+zJH78vVhfZkj+JL1YXvR7P4FpfdEFLTd6IwltR/7pv2xO+zJH8SXqxwrMk/xP8AjB70Oz+A0y7kP7Vd+6H1oGVqu/dD60TlsmT/ABn+MNWxpX8YML3sfn4DRIh/ajv3Q+tDktN37r/KJK2NK/jBjiWOxveGN/Xng97F5+B6JgEtJ37oYMzNTTu5K1eTExiUaa3ClPzCqr7ViYL7/IflPVX5xjPiV/FfJaxvqyDq5/8ADD60cifr5n7+U9vzjsY/Uz7Fe2iYDUPKloKjpEekWCRivsm03f4jSS0iHot0NJ6US/2wh0XswzqnhmZ0t79rfN1PQq3R69HNRcT+l9jy56iUdKfmdmpkh1q39apgnnWKzwSet54ZnSERbk2yRWbNbK9FVNiulyl6kw9sWctLMS4UyjAtj0W27vdEoGHz5NPlYQch0EOaKikMvkwpNp2YOo8oxIYkB/m5vd/uLABHkQrCxANAUx2HXQrokQxUiFaBtS7OtMqRHeiY84LQVRi9L3JmbZ1DRFm3qfYiRMpUVCNimNMpaXMhlGtYW7URXey6Kx/SSZmMwFGcfsj7PZ8JtN/VsVIJFiu3Zfdsi3sUJGbltbJOi4xsq60XFOeI03ua3pNPozPvzFTTvK3eqJ04P7SXk/FbvfFXZSi1MjRFxOLWYlSQ1Cvw+cC2lQpbxsikkcphKsJFjc5xipDVWEZQ1VKuADiwxYffDCSABvZhphHavrZCv6EAzgIUOht0cUi6MAgsKBwoVIC+bkWvuv8AL4QcJQfuhieKbsPRBh2VZECX7IwYWqINcMJSEIQjiDDroGrwxzWwgC3Q0zojglFdPzGR/s/XxhN0rHFW6Ito2vLNHnKr4xWSbnh0y67TSIxUEJTE5UfSjU2XJeDyZFTURZi6/THO7Z1Koox0wLU2zOO0k+642tIjutiqZcVwRVRUVdu3mh1lhqmWmtVqx1SLq9t16Jt645NWw0dqzLQMOMbo5sVwS7BEw5sL12RGZO0ZiZma5V9tohubccbVu9FS9ERFW/jG0eRnLmXlmGNdXaUfbF9OEJyZEHJJPl8Yz0ky6FI70XJAQWbMkeXKnvSM/wCVlNfaQkcjlY9KAIuSoIE09rTyZcvRXnjoOYkV15uTHSMYEilRnji9Lk+2AQc13YbfAr/rnjnIgGPJcnRgZIW7V5MJVryw1UKsv8YADZaKao5+aAqVH/bHBKvc3YAJF/ajkBqjkMDbA7kGHa4elFOj9Ab0COcgodFw5ODyIAUyUVXhPajqTBdKCh0WiPlDkfKKvwgulDDmIAotXJ6gN6AWYYzevE+lFFNTOSDaPzuqnM/KiZrYqPM0YWdLBS7qsxFl+KwN03a4splKwaIN2nLEV5rPV0sYx5cik75lE8E4FqyzuvqlqribFvECVb0JVTanDHZhFuTQmHjREvmkPRqDCzBzG2gSCPRH1Yh2+eqsp3+oSD7b/hFuMvRvxm9LHxPVMZaalP0IifGKXNEt7FI2Rcv5J7I7fX5XZiMFR8rnEqvrGOiY5qOvrvx5+EbGQbWlmoKEr40dmBV0BV9d0DaLWs/uiGq/Lx280AiWhwxXc/Z98BbEQCnk9rDhCFwfJgAK0YhuQie7QxGM8/KHNyfd1w81Ls/m4QDCIRdH/iEbhAHSGro7PlAjIg7WzNA1LOW98+rGAA/hB9CORH1LaYU7IUAFr4R2o6j0UwzEEGYhlFuj0OR2KsZiCI/ABY62BOOxF18AemhDlQhjpl2I7D+qeEgiM7PsBys3uiFM2zJyga2YIhH39ycYTKSPTrCtgXWdU79d0XaiJhvZdolzR4pJabSzL2SWfp6RCl3ovvjY6OfpFs6be8GmBJguTV70v2xi00aywzS1U0bltnterElFEP8A8j8ViHrq+VV7o6b2SFqSMabFOzFARjdKB8TLFyiIvOmHD62xoZp2uM/pjk+zx/pmXpVE+EGN3Kypqo0UAGXS5K/XfCJ2g8nV7u/CBIdZ07va9Ozmjuainldn0p7I6DEJrSzeKLyi2R1XHeWW9mESLBeHo2QIULL/ANvDZxXht88cJaKfJ9OGxL4AH66jxplT5OGNy3e+ELlZ7o9Gr24fPrgKoX72qkauViq47PekJw6MoCNX/HGAAyv0fV/1dA1e8c1mylyeeGG3XT0h78McME+tsMJqsN7Lhu8yY7YADHMEHZERXu288c140Vckt0tuPmgBN9Cmkcd3qww4RxELyvypjcuzqhiJOsDpQohqjd/N1QoABg7BwciG2kSW4bLJQFBUWI4rD6okAqwJWK45XBW1hMpD2bKEwJ2nKIqsec21rXbYd1v8sqRHgmKph6Fj2/R1kZgCaPlDHn/6Q9HSkbSKelxqYezFTwXj7b18680YqT1NHdwujXHUY9gYkTGQNa1VUzcY896cMOEDZLlRodErGO27YYap/Y2yR19zgoot9Kc963JArs+hzyww4ZuT6Hp2jbzpyDTR/wAslDzIuHsi6LcgFny1HJ3iVfSt8TXQjFq22j5XVyK9tut4a+lFJpsY+Hyw/wBKr2rfGml2/HRjtO3S+2xHk6oUq5lvJUw88a4kRkdlJ4SLu4QlThTVivV37IYD5V0mObBc2OHFdmz5QFF3aC6sv+uHshtJAdVRFVuiW27G7HZHQYk1w8m7Vxp4J5080CUR8VmKod7mNVwx6sYSu0ckiGrdEkRVvx48IALhO7hVCRedE28+2/jABJNzIOt3uiOGN8MNawLKPZIscU2r13LwgKnXMtDUVI5c16+ZeHCHVidVdQjgPtgAI2JUFm5W9hcnnTqjrxj5XJ9irfd6YCcwIU62oS3qRvW6/DDuwjmsrDOJUkSU0jdwuxRYACLSB1bpESIJVY3dffHVXOWXd/3DVzgTtQiLfSuxXnTGAuP0M5GiKrdHr6lXZx2wAGx4Nld1bIUAVb1VaCXrpvhQADbSDCkUf6ySYclwvyx1NJ5X7hyGUXyFCrimDSSTPfFwfyxMl7TkZjcfH82EAycMSGBzwNsa9zNEprJEgaGwiJoxIIvbXs9q02c5CJU+aMc1NP7oFEthXT3yIohxRab6EUtBpM5mp12W3vu0Ivd71jU2XZ8tZ7OolGqR5RFvH39XVAZMQizaIQ5UTJtqimyZLNw58YG1MtByojTdqsBlAqi7MRaSJpthRWg6vrqjz7S52vSGZHL4ttumrnuT5xtZWZ8Ie3vVjzzSAmpi2JwnS3nSEcqbBVUu29UPFK2GSNIrnBIwEjqbpK/Vt8blXbdBZep0KQqqxqqxwXrx54YrZZqCq2EVRXXYbUVFxW6BI+QAT4O6siwcGpVQ0vvuW7bHSYBVprLWiNVNOa+/0rBREgZqPd6Xsvv4qiLEVHK2RoGoR3auHmXZDjMgDI7SJCgkJXKirx70whDDuEWags3J23c+2IwuO11OkJDTujxXj3JsxgjMw60eSnNukI7E4XXrgnNhBHHWgCqYJwnSFRGkU2pjety4/wCoABZTqIBpLHe3r9q4+bnhwFXlzCQ35vlfEZshOnVDmIr8o7UTHZtXjjBBdaM90elULipiq3pei+fhAIIPistJZivy89+PHHzQJC1rxZcu7VVjfj7IGY1mRcneLxl6X4cOCqqJBpk2DkxaayvjcREI3IqX4pjffDGOUpYVVFcxTBcsKImrmFxQ1RF4LfhCgEUQWYnPBksxIUKGWIrNSAO2eIQoUAzjUzNyJ+ImC8ksUjQWZpSjnipoFE+kOz0QoUICzXSGWa6XqwwtLUDcBYUKIGDLTF7kpDU0qmT5ZRyFCaRaLCStWZmjEdaUQdIdJtVVZ1mEWvqpfmMUVLtopf7V9G2FCjKMVZUm6LzQO1zUPGZyHnjPTc/XPFMUlU66pKglsqJcMe/2QoUGL82Tl/FHFm2kJpuZqIEFRpEUwTq5oMFpCrOobEW6TvytpsTaiou3ai33woUdBzkcZtgtZrhJx3ZVsRMUXYnmhOEIPFqt7lVDjjguKL1rChQwFU484TIOlcLa4FsTqT2wMTfOtp1yoVVLqcKbl2de2OwoQHDaWXdcU2hJW7r0qx9MCaOpb91RxuHhxhQoYgjLqtgJsFcRXpUoot/Pci7OMIZIp4Dc15CLAopF1X3rhx2QoUA0M1NWYXipXFL0TZ6IUKFDA//Z',ft_model)

1/1 [==============================] - 2s 2s/step
Validating location of damage....Result: Rear
Severity assessment complete.


In [31]:
pipe3('data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBwgHBgkIBwgKCgkLDRYPDQwMDRsUFRAWIB0iIiAdHx8kKDQsJCYxJx8fLT0tMTU3Ojo6Iys/RD84QzQ5OjcBCgoKDQwNGg8PGjclHyU3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3N//AABEIAMAAzAMBIgACEQEDEQH/xAAbAAACAwEBAQAAAAAAAAAAAAADBAIFBgEAB//EAD4QAAICAQIEBAQEBAMGBwAAAAECAAMRBCEFEjFBE1FhkRQicYEGMkJSI0Oh4RWSwTNTYrHR8AckVGNygvH/xAAYAQEBAQEBAAAAAAAAAAAAAAAAAQIDBP/EACMRAQEBAAEDAwUBAAAAAAAAAAABEQIDEiExQWEEExQiUTL/2gAMAwEAAhEDEQA/APl7GAcwrwLdZ53qqDQR6wrYxAMzEjCbMdjNxztWOm3oT7wGpbNuPKT0bsNKxYYwSIA7tzecSeUtSAPUdo6tYrp26mL6YAuMxm5x4LAd5WARCLI01liMRxaVJ9ZcQJGKdMb+cIj8x5WAOfKNJobHG1Tya8PuDDFR+4jsTQOUKMAYnCJYDh+oJ2rHuJMcN1J/lj/MJOyndFUVg7BLk8L1P+7H+YQVvCtSd/Czj1BlnC/w7opsT2JYPw/UL1os/wAkXahl/MpEvaui8L/2rDzE9xZsui+W8HUxpYsuOmN4LU2PdZzWY9MTl2+dalBduYg/aOaXXpVStbITjuImFzJCuLx0lWHxNdzYGcjzit6fxh6jMig5GU+sNqRi1Gmcxou6cykjqsVIyc+fWPqeRx69fpFdRV4drL26iUDfrBPC2GBaajpyoL4IKnvICzmCeJSGK7DfEk8Jp15rVHlvNORnUNy04xjMVCZIEPq9yo+8GoxvEZo9GBU7HucTtKc2579BOUB7F8KsZOcyy4ZpvE1IJGFXczXHjtZtwzo+E/IGuYnvy+UtaaK6hipQo9JNR37QgE9M4yOFtrqiTAngJMCVHQJNROASQEKmBCASAEIIRICS5FcYZQ31GZESYIHWRVbxTR6UeHz6dCASzBFwcCZ7ium09b/wB0UZIOxM0OrtBtew/kUbj0H95l9dZzPy+ZyR5Thzd+BBlYfecXnBhnkEdlzy9+s5OmD0VHHMe87qdgn1gltYAADI7T1lhcYIxM1XnOLB6mF8LxFX0GIKtDaQc9No4unbHTMMqd4E9YdhBHAO81G6CwPbrHaKRVVzH8zdfSR0VPiMWIJCHp5w7uDkfqzgDylTCmo3cTgG0Zs0zMwZRnaQNTgHKn2mpGOSOnLV3CxOqzR8JT+AXP5nOftKTS0l3VSCAesvKc1gBCQB0xOvTjlzvsslhQMxFdRaBg4hF1L+QnZyw6qwgERGpbuohBqm/aIMOASYESGqb9okhqj+0e8hh0bTzutY5nPTtFBqmxsgg7LGsI5u3SFwS3XlFLYAA6A94n/jrsHAVAV2zOauk30lFblfIKntkSl12hZF57EVWbJPKczNuNyGruKI9bIWRc7k5zKt9RSN2sJJ74gPhwfMznw4H6feca6yCHVVD8rE/aDOqr+n0E4aV/aJHwvJZmr5SOqXtk/aeGoVjjG/0kfCI7Y+06KyN+Uk+gmaeRcnqNvtPG1x/Mb3gytrD8rSHg3H9LSYrrdJPR6VtZqVpRlQnJLOcAADJJ+0gdzG+F6Y6nUlF/bG46etM6epaqwULLkDI84O3Tr4mV6HeXul0HjHkHh4UHm52wFH/YjF3CObT+IrHnTc1+H19OscfLfP9Z2qSmjaHXT57RyijmUEDbEZTTkHpOsjz2qz4b0hV057S7o0Rbosfq4VkbribkYtZoaZj2MkNK3kfeateFL3GYReF1jtNdtTYyXwr9g0l8LZ5NNinDax+mHTQ0r1VYz5N+GJXR3n8qsYRdBqz+WtpuFoqHQLt6TxelOrKBJ4/p6+zGJwjiDdE6+Zk7OCcQrUF+UBjgZaX3EOM1adT4OGYeUBorLtVYuo1Rwv6EP6fWZtak+FNfwTiVX5qifVWyIlbw3UHa2s/cTeLqFb5FDMPPHSTNBK52x6y+qej5w3D8fyunpBHS/+2PafQbtMhBzWCZW36Wk5ygBmcblYs6U9kHtBvp7AMqo9pqbNEvaKWaQCZwZ2uq3n+cD2hvD8h7S0s04EXNeJmxuEjXBsm8dYYgWG8mKz/eXf4XrB1L2scKoBY+mZR95rfwToW1lPEDWru9Yrwi/rBfBHsTLiblaq3gelXhnxensdntfdDuB1b+8BpXrTnXU8yNWAPPmEjoLrtBqqUu51qDjxac7HzGPoZb6bhtGi4hQ948dCSCCflOD6+omuPGey8758qTT6ev4llUYrswUz5Rv/AA8oxBEuPxXXo6btO2iavlUAFax08v8AnJ6dfGrD7E951kcOVA0Gj3VQQpbbLHAljXpeYfLdQ2xOfEGNtsD12kqUwwzjbeEvNbNzNp6SQwY9dyM+f19JLzk8M9uh+EEXJavGOgcZ6Zx9YN2pqVhZfUGTHy8xzv0+sQ1WtrCJU1WnKqrA/KCTnz+krNZqabCPD02mUdNq1B98fSO9qcFrdrVzhGDbZzFzq2zttKddQQijCjAxhegnm1LY2mLW5FpZqm5d35fTMrNXqjjCnOYu15bqYHPiPyjc+kyomnoN9o5twNzNZoNOFqHTJGwMq+GaUAjPU9ZoKq8VqR1TpAGPlQkjYdcbSw0l9VyGuhWS4J/H0Vy72r5o3XMr772rVhWeUncHPSQr1ms8IJqcaoK3NRbYcPUfqOom5fDOeUXKliELcuSAXO/3iV9JJOMQ91jPc9jks7nLHGIB7I1cI3VMoOR7RGwS2NqkEEjeVWowGIXoJKQlaIpYI3YYpYZGiz9QPWcK1gnm3PoZ2yBJ3hpnCQB1E3H/AId6pNN/iAa0I1laeHkbMQ6nHptmYetTY/KpAJHUnEvuDFtPp7c2FeVAXAGz/Ou3vg/aJWc3w+pvwmvi91eo+NoXKglHsAYZ7YnOM8+laqsrkFRgg45SMD/QH7zNcM4qNWAdVaXpwOZlGSD6j/WaLg2uBpfSU22FfEDVsrbEdwQdm7bGXjZEvG+6k1ZFrkqwxghsbH2/1lvwO82VcjH5l2Mt7uC8O1eTbV4Fp356PlH3Tp7YlW3A9Tw3UC/R316ivoy71Pj6HI/rLqX9vB260VHEq+Ia5xUcNiOWONTQQp+cdBKHXLYz8vK2B9pOU8oQvscOTknMXa589Ye3UKdR8OamJBAzzf2ls3AaVId7yBnBATpjz3nn6vW4dP8A1W5wtUeWxnM9lj3mho4CtqMy2WfIehrxn3MZo4DWSvM5AY43UDsT5+k5/ldK+7XZWZr072HAzLPS6NKSCRlpoF4EgPyOR36RTWcOfTMD4mc/0nTp/UcOdyVLxsc0q4xLZT8gER0NYrw9hBzH7QnLzA/YTrZrJHXDkA9TIWOQiwWqt8S3lByBBWWgnc7eko9dbvA+IMnm8pxtQqnCLnHn0iN9xLEnEArWKDEL3BY4njZkxW2zcy2kiFjdYrY0nY8WdoacY5gW6yQyx5R1kShB3Ye8jTOVHlsUy201vKd1BB7HvKuoDxFB6noJYoN1YbiRzWWjqcsp017Usu3XIP2mg4dqdfpLVus0fjFWzz0kAn7CZ7QvytNRw+4EKCZYs53Gp034s4bYVXUF9NZjdLVI+8t6dVpdSM03VWqf2MDM9XUllfK6q3o28Xs0mlrObKkBH/Dv/SbT9ac4mBotdz1nFdm/KfPvI6iuu1A6Y38pU3ihmBrXUcyn5SbScfQEyNrankCVPgqPyvkH65H/AEjWbITTRfE8csXn5AmXzy56Y2mktFQt15fVWLqVP8Gvn3Y8oxhe+8ytja7xSOYFifzbZ98Zlkj8cSjnTUh1xspczy9f6f7l1049TJi7sfUj42xA6FXRedbCPD+Ub8o/MATmMCy1Wv1XiFq6NThgo2NRUA4HpnP2I7zOLxTjC/KTnH/F/eGXjGuQc9qvt69f6zj+Jc9V+4u7r9QBo7NRca/iXsdla7wwi4+VebHYY+5MJqCl+nrVLA+Qfm8Tn/rF11HEdgK93HMPnG594tdTxTUMxbTfONl53Ax69Z06X0/ZynJOXLZhuipgOQtjHT1k7OetGLNnb2laRx5UA+FqsK/qFggs8YsGLNMeU9QhWeuyOe1FrMk4O+YOx/1Qdun14DMtJCjYksu0XfScTPL8oUMCVJZN5MXU7nwN85iNlm//AFk7dDrsI1uopQOMrzWruPMQdnDL1Ki3W6ZOYZGXO49oNBazAz3gHYnJlgfw+/xNdL8RqWx+gBPfpIX8C0tdz03cRbxlyOUUnc/WDVS7QDOD3EtG4LSAM3McjtiLvwerIILEephdV4dOcc7fL6dZzx6BsAx+omi0XA9ISDZWW+plmvB+HoMDR0/dBNTinc+TV2f+bq8g43lvWcHl7CUKHB5vI5ly5KucHqAR65mElXGkQEgzScOpJ5SBt3Mx+j1prI+XP3mk4dxqta+VkYH0M1BsKfDRF+YEfXeEsOnvABC575lHTxbTsBuR/wDWMpxDSt/M/oZe3incsF01dZzWAo8x3iGqJawsBlh29JIcU0q7eOsA2v07OTzggjzk9FV9xNdu/wBQfSXHC7Vsq8NiM+UqdVZTYP4TDI6b9Zzh2pNdq+pmotWuoqC2bDAMEKg2QR2MeuAsXI3x0xF0Q5JYY2P/AChl9B/Ci7Xlq1HyV4bz/NL8qpOSoJ9RMHouPVaWlArXg8ihgtZ7SLcbwij47W5UuQwAyc9M7dpMG95EHRF9p3w0GeVFH0EwK/iJq6VRdVq3wMc1igk/XpOU/iU6dcG/VW+rY9owb/w0xui79dpzw0/YvsJ8/P4oPzDm1nzf8QGPpIH8VsOUAao46EsJMH0M1Vn9CnHTKjae8Ks/y08uk+dn8W3AbLqP84g2/GOqH6Lfu4gfR/CQnetPriC11aHR6j5Bnw27d8GfOW/Gut/Y+e3z/wBoG78Z8SsrZDU2GBB+f+0oyrMfado+ezB6TxrfP5TvGdHpibOYjAjFh7TpyiF3M8fy7DptK7V63S0W8mpvSt8ZAPlOk8M18lAztLFXNlSP3xy+0rwCDnEZ0j/KyntuJwNNI+DHqbPWVfNhozS8RpcU6ll7xpdU2MZxKcOSvy4zD0s4X58Z9JUWauWO7ThsZe8Wrs3G8m1gDfNCieK2cg9IzTq1/mkh/wB4GYlzqekizSi9r4xqq15an0rKOhY4MIvG9ceiaU/RpnC2Ok4X85dRpxxfXdqKifQz3+L8Q7aao/X/APZlzaSCN5Gq1mXLAqfIy6jUHivEj001Q+395A8T4n/6ev2/vM8LPWdNnrGqvjxLih6UVD7f3kTxLifemkfYShNg8xIGwecai9biXERswpGfpBPxDWjrdQv3Epefykecyaq3biGr76uofQwD6689daPtvKxnMFlucNnbyjRZNrXHXVOfoDIfHFTkXXn74Ery3WQLxothxzVVKRSxGdizHJH0lZbc1jl3Ysx6lupgS8Ez7xbQu1DDrvIqnI3SOuJ4VA1s47DaYTC+ATkQtanyi6E5K5x5ZjNNrqMESqYVWwO0MCQOsgmpQfmUwy20H9WPqIHA7Ag4k+cscmdDUn9YhUFXd194EATJb4hQtJ/mL7yfJUP1r7wF95zBjSLUc/OvvJBKs/nX3gJ8h8pE1uegMs1FP7195MeCP1p7wuKjkcec5hh1lqwpPV1g2Sju6xpis38pw58pYldN/vFg2+GH8xZUwjv5TmDHSdOP5i+0Gz6cfr9hIFCCJAgmNNbQOhJ+0G19I6ZgLFD5yHhnzh31FY6KTAtql7LiBA1nzkeSebUn9Igze574gOOh32h9TWKtMqj8zDpDaevntHkNzO31rdY3Pcq8vyjMgoraiTsN53nbADgqR5yxt0ta7jUKPoYnao72q2/eUDVj2OYRbMwRCjbxAP8A4iTRa+Uc1x9o1BQwM6bSvSQxSOtre04PAz/tW9pQZbSRnBxOfEt2Eiqj4dmVmIEXZk7MxEgaGpbuMzo1J/b7RMMO2fvJK678xP2lDZ1Xp7znxZ8gIoz19maDLJ5k/WTYH/jCPL3nG1xA6DP1iBdQMg4x6SKay5D/AAWAHfKxrfD5izGsXkyR83pBfF2NnlUem0h8ZqAgLWnf9qiDbV3MpxqLR6ZkdLy4+Jgg1N2/OOWcOoY9TE2LE5ZiT6yBz6S6404bie8ibD+6LVuQwBAIMcK04Hye0aWBG3zbaSOy5zsZILSDvWcQlQUNyqPlPaGSxJzPbx0tQuz0kkd5NbNMBvpz/SBe6cCjRPedmYZlVYMkls5l3xYhKq9OoxkDI8gNv+/pKllzCk3TY7RV6/mGNvtLJkB2izJgnPbpASKfN1zCKkYFXNuu4hUox1EIVNW3SR8LG+JYeDtsJHws7YgRoVfhHB8ol4Ylz4KDSkYyYk1XtIpPw541iN+HPeFAQav0g2Q9hLB6ts4gbKSy7dYIrrGGeXMYr0nNVzB0HoT1k9Nw97m5mGFz37yzTQ0IMeGp9TLJr0dvjC9Oka2nlTkz9YJ+F2oMsV/zS50qoMhAox+0YguLVlqQFznPnFi/a91G9DAkErt5GCavH6gJO1dz/rAqhzgLkmZ1L0nUALqAc7yzNYwIrptOwfms7dJYDDD1iOHP4LGvfaSrTDZhwk6EPlKxEHqDqcDJG8GEOBHKlGQDJPp8tlekK//Z',ft_model)

1/1 [==============================] - 1s 713ms/step
Validating location of damage....Result: Rear
Severity assessment complete.
